# Get Data

In [89]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly_express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

In [101]:
stocks = ["AAPL", "MSFT", "NVDA"]

In [102]:
class Stocks:
    def __init__(self, ticker_list, start_date='2020-01-01') -> None:
        # static variables
        self.today = pd.to_datetime('today').date()
        self.start_date = start_date
        self.ticker_list = ticker_list

        # data
        self.df = self.__build_df(self.start_date)
        self.sp500 = self.__getData('^GSPC', self.start_date)

        self.df_pct_change = self.df.pct_change().dropna()
        self.sp500_pct_change = self.sp500.pct_change().dropna()


        # calculated results
        self.GBM = None



    # private methods
    def __getData(self, ticker, start_date):
        data = yf.download(ticker, start=start_date, end=self.today, interval='1d')
        return data['Adj Close']
    
    def __build_df(self, start_date):
        df = pd.DataFrame()
        for ticker in self.ticker_list:
            data = self.__getData(ticker, start_date)
            if data.empty:
                print(f'No data found for {ticker}')
                continue
            data = data.rename(ticker)
            df = df.merge(data, how='outer', left_index=True, right_index=True)
        return df
    
    

    # public methods
    def GeometricBrownian(self, ticker, start = '2024-05-01', iterations = 1):
        stock = self.df[ticker][start:]

        Mu = self.df_pct_change[ticker][:start].mean() # expected return
        Sigma = self.df_pct_change[ticker][:start].std() # standard deviated of the stock returns 
        dt = 1 # daily
        T = len(stock)-1 #-1 because the first day is not being predicted
        N = T/dt

        simulations = {}
        for i in range(1, iterations+1):
            S = [stock[0]] # initial stock price
            for _ in range(1, int(N)+1):
                S_prev = S[-1]
                # drift - longer term trends
                X = (Mu - 1/2 * Sigma**2)*dt
                # diffusion - shorter term random fluctuations
                Y = Sigma * np.sqrt(dt) * np.random.normal(0,1)
                St = S_prev * np.exp(X + Y)
                S.append(St)
            simulations[i] = S

        self.GBM = pd.DataFrame(simulations, index=stock.index)


        
        # Create an empty figure
        fig = go.Figure()

        # Add all the simulation traces in grayscale
        for column in self.GBM.columns:
            fig.add_trace(go.Scatter(
                x=self.GBM.index,
                y=self.GBM[column],
                mode='lines',
                legendgroup='Simulations',  # Group all simulations under one toggle
                name='Simulations',  # Name that appears in the legend
                showlegend=False  # Only show one legend entry for the group
            ))

        # Add one trace to represent all simulations in the legend
        fig.add_trace(go.Scatter(
            x=[None], y=[None],  # Dummy trace for the legend
            mode='lines',
            line=dict(color='gray'),
            legendgroup='Simulations',
            name='Simulations',
            showlegend=True  # Show this single entry in the legend
        ))

        # Add the actual stock price trace as a seperate legend
        fig.add_trace(go.Scatter(
            x=stock.index,
            y=stock,
            mode='lines',
            name='Actual Stock Price',
        ))

        # Set the layout
        fig.update_layout(
            title=f"Simulated Stock Prices vs Actual Price for {ticker}",
            xaxis_title="Date",
            yaxis_title="Stock Price"
        )
        fig.update_xaxes(range=[self.GBM.index.min(), self.GBM.index.max()])
        fig.update_yaxes(range=[self.GBM.min().min(), self.GBM.max().max()])

        fig.show()

        



var = Stocks(stocks)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [103]:
for ticker in stocks:
    var.GeometricBrownian(ticker, iterations=100)